<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Inference

In [2]:
import os
import sys

import numpy as np
import tensorflow as tf

from deepcell.applications import NuclearSegmentation, CellTracking
from deepcell_tracking.trk_io import load_trks

sys.path.append('..')
import utils

In [3]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

In [4]:
data_dir = 'ctc-data'
gt_seg_dir = os.path.join(data_dir, 'SEG_GT')
pred_seg_dir = os.path.join(data_dir, 'SEG_PRED')

for d in [data_dir, gt_seg_dir, pred_seg_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

In [5]:
files = ['../../data/Fluo-N2DL-Hela-test/Fluo-N2DL-HeLa-test-01.trk', '../../data/Fluo-N2DL-Hela-test/Fluo-N2DL-HeLa-test-02.trk']
X, y, lineages = [], [], []
for f in files:
    data = load_trks(f)
    X.append(data['X'])
    y.append(data['y'])
    lineages.append(data['lineages'][0])

data = {
    'X': np.stack(X, axis=0),
    'y': np.stack(y, axis=0),
    'lineages': lineages
}

In [6]:
# Load segmentation and tracking applications
app_seg = NuclearSegmentation.from_version(version='1.1')
app_trk = CellTracking.from_version(version='1.1')

INFO:root:Checking for cached data
INFO:root:Checking NuclearSegmentation-8.tar.gz against provided file_hash...
INFO:root:NuclearSegmentation-8.tar.gz with hash 507be21f0e34e59adae689f58cc03ccb already available.
INFO:root:Extracting /home/msschwar/.deepcell/models/NuclearSegmentation-8.tar.gz
INFO:root:Successfully extracted /home/msschwar/.deepcell/models/NuclearSegmentation-8.tar.gz into /home/msschwar/.deepcell/models
2024-07-09 16:01:09.400563: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-09 16:01:09.533327: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46873 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus

INFO:root:Checking for cached data
INFO:root:Checking NuclearTrackingNE-8.tar.gz against provided file_hash...
INFO:root:NuclearTrackingNE-8.tar.gz with hash 79188d7ae32b40b5bd0ad0f2ac2b53c4 already available.
INFO:root:Extracting /home/msschwar/.deepcell/models/NuclearTrackingNE-8.tar.gz
INFO:root:Successfully extracted /home/msschwar/.deepcell/models/NuclearTrackingNE-8.tar.gz into /home/msschwar/.deepcell/models


INFO:root:Checking for cached data
INFO:root:Checking NuclearTrackingInf-8.tar.gz against provided file_hash...
INFO:root:NuclearTrackingInf-8.tar.gz with hash ec07d8c0770453e738f8699ceede78e7 already available.
INFO:root:Extracting /home/msschwar/.deepcell/models/NuclearTrackingInf-8.tar.gz
INFO:root:Successfully extracted /home/msschwar/.deepcell/models/NuclearTrackingInf-8.tar.gz into /home/msschwar/.deepcell/models


In [15]:
for batch_no in range(len(data['lineages'])):
    print(batch_no)

    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineage = data['lineages'][batch_no]

    # Correct discontiguous tracks, which are not allowed by CTC
    y, lineage = utils.convert_to_contiguous(y, lineage)

    # Determine position of zero padding for removal
    slc = utils.find_zero_padding(X)
    X = X[slc]
    y = y[slc]

    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    X = X[:len(good_frames)]
    y = y[:len(good_frames)]

    # Generate tracks on GT segmentations
    track_gt = app_trk.track(X, y)
    track_gt['y_tracked'], track_gt['tracks'] = utils.convert_to_contiguous(track_gt['y_tracked'], track_gt['tracks'])
    utils.save_ctc_res(gt_seg_dir, batch_no + 1, track_gt['y_tracked'][..., 0], track_gt['tracks'])
    utils.save_ctc_gt(gt_seg_dir, batch_no + 1, y[..., 0], lineage)

    # Generate tracks on predicted segmentations
    y_pred = app_seg.predict(X)
    track_pred = app_trk.track(X, y_pred)
    track_pred['y_tracked'], track_pred['tracks'] = utils.convert_to_contiguous(track_pred['y_tracked'], track_pred['tracks'])
    utils.save_ctc_res(pred_seg_dir, batch_no + 1, track_pred['y_tracked'][..., 0], track_pred['tracks'])
    utils.save_ctc_gt(pred_seg_dir, batch_no + 1, y[..., 0], lineage)

0


INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Tracked frame 1 in 0.40225896798074245 s.
INFO:CellTracker:Tracking frame 2


INFO:CellTracker:Created new track for cell 65.
INFO:CellTracker:Detected division! Cell 65 is daughter of cell 5.
INFO:CellTracker:Tracked frame 2 in 0.35272837919183075 s.
INFO:CellTracker:Tracking frame 3


INFO:CellTracker:Created new track for cell 67.
INFO:CellTracker:Detected division! Cell 67 is daughter of cell 65.
INFO:CellTracker:Tracked frame 3 in 0.3535239868797362 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 68.
INFO:CellTracker:Detected division! Cell 68 is daughter of cell 9.
INFO:CellTracker:Created new track for cell 69.
INFO:CellTracker:Detected division! Cell 69 is daughter of cell 13.
INFO:CellTracker:Created new track for cell 70.
INFO:CellTracker:Detected division! Cell 70 is daughter of cell 13.
INFO:CellTracker:Created new track for cell 71.
INFO:CellTracker:Detected division! Cell 71 is daughter of cell 9.
INFO:CellTracker:Tracked frame 4 in 0.35425032395869493 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Tracked frame 5 in 0.10237803612835705 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Created new track for cell 72.
INFO:CellTracker:Created new track for cell 73.
INFO:CellTracker:Detected division! Cell 73 is d

1


INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 101.
INFO:CellTracker:Created new track for cell 102.
INFO:CellTracker:Created new track for cell 103.
INFO:CellTracker:Detected division! Cell 103 is daughter of cell 78.
INFO:CellTracker:Created new track for cell 104.
INFO:CellTracker:Created new track for cell 105.
INFO:CellTracker:Created new track for cell 106.
INFO:CellTracker:Detected division! Cell 106 is daughter of cell 4.
INFO:CellTracker:Created new track for cell 107.
INFO:CellTracker:Detected division! Cell 107 is daughter of cell 4.
INFO:CellTracker:Created new track for cell 108.
INFO:CellTracker:Created new track for cell 109.
INFO:CellTracker:Created new track for cell 110.
INFO:CellTracker:Created new track for cell 111.
INFO:CellTracker:Tracked frame 1 in 0.14749058801680803 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Created new track for cell 113.
INFO:CellTracker:Created new track for cell 114.
INFO:CellTracker:Created new tr

# 2. Evaluation

In [16]:
import glob
import os
import re
import subprocess

import numpy as np
import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [17]:
data_dir = 'ctc-data'
gt_seg_dir = os.path.join(data_dir, 'SEG_GT')
pred_seg_dir = os.path.join(data_dir, 'SEG_PRED')

pattern = re.compile('\d{3}_GT')
data_ids = [f.split('_')[0] for f in os.listdir(gt_seg_dir) if pattern.fullmatch(f)]

node_match_threshold = 0.6

ctc_software = '../CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In [18]:
benchmarks = []

for results_dir, s in zip([gt_seg_dir, pred_seg_dir], ['GT', 'Caliban']):
    for data_id in data_ids:
        results = {
            'model': f'Caliban - {s}',
            'data_id': os.path.splitext(data_id)[0]
        }
        gt_dir = os.path.join(results_dir, f'{data_id}_GT/TRA')
        res_dir = os.path.join(results_dir, f'{data_id}_RES')

        # Deepcell benchmarking
        m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir, threshold=node_match_threshold)
        results.update(m.stats)

        # CTC metrics
        for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
            p = subprocess.run([os.path.join(ctc_software, operating_system, path), results_dir, data_id, num_digits],
                               stdout=subprocess.PIPE)
            outstring = p.stdout

            try:
                val = float(outstring.decode('utf-8').split()[-1])
                results[metric] = val
            except:
                print('Benchmarking failure', path, results_dir, data_id)
                print(outstring.decode('utf-8'))

        benchmarks.append(results)

df = pd.DataFrame(benchmarks)
df.to_csv('ctc-benchmarks.csv')

missed node 1_23 division completely
missed node 3_29 division completely
missed node 4_20 division completely
7_5 out degree = 1, daughters mismatch.
10_5 out degree = 1, daughters mismatch.
missed node 19_10 division completely
missed node 27_22 division completely
missed node 46_20 division completely
56_37 out degree = 1, daughters mismatch.
missed node 62_5 division completely
missed node 79_73 division completely
missed node 80_78 division completely
missed node 83_54 division completely
missed node 97_34 division completely
missed node 386_76 division completely
missed node 107_46 division completely
missed node 417_51 division completely
missed node 120_60 division completely
missed node 126_76 division completely
missed node 419_60 division completely
missed node 131_74 division completely
missed node 144_73 division completely
missed node 175_90 division completely
186_81 out degree = 1, daughters mismatch.
missed node 255_63 division completely
missed node 300_84 division co

In [19]:
df

,model,data_id,correct_division,mismatch_division,false_positive_division,false_negative_division,total_divisions,aa_tp,aa_total,te_tp,te_total,DET,SEG,TRA
0,Caliban - GT,001,133,4,8,24,161,13455,13508,13891,13944,1.000000,1.000000,0.999344
1,Caliban - GT,002,93,12,18,28,133,17183,17465,17667,17949,1.000000,1.000000,0.998996
2,Caliban - Caliban,001,120,7,14,34,161,13424,13508,13859,13944,0.993768,0.896498,0.993366
3,Caliban - Caliban,002,93,11,14,29,133,17129,17465,17610,17949,0.993638,0.896924,0.992289
